In [ ]:
from datetime import datetime
from datetime import timedelta  
import numpy as np
import pandas as pd
import random
import time

# Filtering for a month

In [ ]:
def validate_pick_up(row):
    if (row['pickup_latitude'] <=  40.66345 and row['pickup_latitude'] >=  40.63596) and (row['pickup_longitude']<=   -73.7664489 and row['pickup_longitude'] >=   -73.8132848):
        return True
    else:
        return False

In [ ]:
def validate_drop_off(row):
    lat = row['dropoff_latitude']
    long = row['dropoff_longitude']

    if((lat > (-0.377522883*long + 12.75605988)) and (long < ((lat - 236.6784444)/2.648846056)) and (long > ((lat- 236.8364875)/2.648846056)) and (lat < ((-0.377522883)*long + 12.9717472))):
        return True
    else:
        return False

In [ ]:
def drop_irrelevant_columns(filtered_data):
    filtered_data = filtered_data.drop(columns = ["payment_type","fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","RatecodeID","store_and_fwd_flag","VendorID", "test1", "test2"])
    return filtered_data

In [ ]:
def filter_data(df=None, path_to_file=None):
    
    if df is None:
        df = pd.read_csv(path_to_file)
    
        df['tpep_pickup_datetime'] = [datetime.strptime(pickup_time, '%Y-%m-%d %H:%M:%S') 
                                         for pickup_time in df['tpep_pickup_datetime']]
        
        
        df['tpep_dropoff_datetime'] = [datetime.strptime(dropoff_time, '%Y-%m-%d %H:%M:%S')
                                          for dropoff_time in df['tpep_dropoff_datetime']]
    
    df= df.sort_values(by='tpep_pickup_datetime')
    
    boolean_series = df.apply(validate_pick_up, axis = 1)
    filtered_pickup = df[boolean_series]
    boolean_series = filtered_pickup.apply(validate_drop_off, axis = 1)
    filtered_data = filtered_pickup[boolean_series]
    

    filtered_data = filtered_data.loc[filtered_data['total_amount']<500]
    filtered_data = filtered_data.loc[filtered_data['total_amount']>0]

    filtered_data['time_diff'] = filtered_data['tpep_dropoff_datetime'] - filtered_data['tpep_pickup_datetime']
    filtered_data['time_diff'] = filtered_data['time_diff'].apply(lambda x: x.seconds/60)

    filtered_data = filtered_data.loc[filtered_data['time_diff'] <=180.00]
    filtered_data = filtered_data.loc[filtered_data['trip_distance'] < 100.00]
    filtered_data['trip_distance'] = filtered_data['trip_distance'] * 1609.34
    filtered_data = filtered_data.loc[filtered_data['trip_distance'] > 18000.00]
    return filtered_data





In [ ]:
df = pd.read_csv("yellow_tripdata_2016-04.csv")

In [ ]:
def generateRandomBoolean(x):   
    return random.getrandbits(1)

In [ ]:
def generateRandomNoOfPassengers(x):
    return random.choice([1, 2])

In [ ]:
df = None

#initial filtering
filtered_data = filter_data(df,'yellow_tripdata_2016-04.csv')
#drop irrelevant columns
df = drop_irrelevant_columns(filtered_data)
#assigning random willingness to walk
df['isWillingToWalk'] = df.apply(generateRandomBoolean, axis = 1)
#assigning random passenger count
df['numberOfPassengers'] = df.apply(generateRandomNoOfPassengers, axis = 1)

In [ ]:
df.to_csv('yellow_tripdata_2016-04-filtered.csv',sep=",");


# Filtering for specific date and time


In [ ]:
# filter on basis of date_time
def filter_on_start_datetime(start_date_time, stop_date_time, df):
    
    df['tpep_pickup_datetime'] = [datetime.strptime(pickup_time, '%Y-%m-%d %H:%M:%S') 
                                         for pickup_time in df['tpep_pickup_datetime']]
        
        
    df['tpep_dropoff_datetime'] = [datetime.strptime(dropoff_time, '%Y-%m-%d %H:%M:%S')
                                          for dropoff_time in df['tpep_dropoff_datetime']]
    

    if start_date_time and stop_date_time:
        df = df.loc[(df['tpep_pickup_datetime'] >= start_date_time) 
                    & (df['tpep_pickup_datetime'] <= stop_date_time)]
    return df

In [ ]:
#Don't execute this cell now


start_date_time_str = '2016-04-17 08:00:00'
start_date_time = datetime.strptime(start_date_time_str, '%Y-%m-%d %H:%M:%S') 

stop_date_time_str = '2016-04-17 22:00:00'
stop_date_time = datetime.strptime(stop_date_time_str, '%Y-%m-%d %H:%M:%S') 

#load monthly data
monthly_df = pd.read_csv("RideSharingData/filtered_data_Apr2016.csv")
daily_df = filter_on_start_datetime(start_date_time, stop_date_time, monthly_df)

In [ ]:
daily_df.to_csv('RideSharingData/filtered_data_2016-04-17.csv',sep=",");